In [1]:
# Import librairies

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import plotly.figure_factory as ff

In [2]:
# URL of the CSV file of INSEE data
insee_url = 'https://medical-deserts-project.s3.eu-north-1.amazonaws.com/insee_clean.csv'

# Read the CSV file from the URL into a DataFrame
insee_df = pd.read_csv(insee_url, sep = ',', encoding='utf-8')
insee_df.shape

(38590, 90)

In [3]:
# Remove useless columns
insee_df = insee_df.drop(["APL aux médecins généralistes de 65 ans et moins", "APL aux médecins généralistes de 62 ans et moins"], axis=1)

# APL column at the end of dataset
insee_df["APL aux médecins généralistes (sans borne d'âge)"] = insee_df.pop("APL aux médecins généralistes (sans borne d'âge)")
insee_df.rename(columns={"APL aux médecins généralistes (sans borne d'âge)": "APL"}, inplace=True)
insee_df.shape

(38590, 88)

In [4]:
insee_df= insee_df.drop_duplicates()
insee_df.shape

(34760, 88)

In [5]:
# Select only numerical columns for the correlation matrix
numeric_columns = insee_df.select_dtypes(include=['float64', 'int64'])

# # Calculate the correlation matrix
corr_matrix = numeric_columns.corr().round(2)

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                   x=corr_matrix.columns.tolist(),
                                   y=corr_matrix.index.tolist())

fig.update_layout(height=2400, width=3200)
fig.show()

In [6]:
# drop one of the 2 perfectly correlated features
to_drop = ["Nb Entreprises Secteur Services", "Nb Entreprises Secteur Commerce", "Nb Ménages", "Nb Résidences Principales", "Nb Occupants Résidence Principale", "Nb Création Commerces", "Nb Création Enteprises", "PIB Régionnal", "Nb de Commerce", "Nb Santé, action sociale", "Population en 2014 (princ)", "Pop 60-74 ans en 2014 (princ)", "Pop 75-89 ans en 2014 (princ)", "Nb Logement Secondaire et Occasionnel", "Nb pharmaciens Libéraux BV"]
# Remove columns to be dropped
insee_df = insee_df.drop(columns=to_drop)
insee_df.shape

(34760, 73)

In [7]:
to_keep = ['Nb Omnipraticiens BV', 'Nb Infirmiers Libéraux BV', 'Densité Médicale BV', 'Dep Moyenne Salaires Horaires', 'Dep Moyenne Salaires Employé Horaires', 'Dep Moyenne Salaires Ouvrié Horaires', 'Reg Moyenne Salaires Ouvrié Horaires', 'Capacité Fisc', 'Moyenne Revnus fiscaux', 'latitude', 'longitude', 'SEG Croissance POP', 'Urbanité Ruralité', 'Dynamique Démographique BV', 'Environnement Démographique', 'Seg Cap Fiscale', "Catégorie commune dans aire d'attraction des villes 2020", "Tranche détaillée d'aire d'attraction des villes 2020", 'Libellé degré de densité', 'APL']
insee_df = insee_df[to_keep]
insee_df.head()

,Nb Omnipraticiens BV,Nb Infirmiers Libéraux BV,Densité Médicale BV,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Employé Horaires,Dep Moyenne Salaires Ouvrié Horaires,Reg Moyenne Salaires Ouvrié Horaires,Capacité Fisc,Moyenne Revnus fiscaux,latitude,longitude,SEG Croissance POP,Urbanité Ruralité,Dynamique Démographique BV,Environnement Démographique,Seg Cap Fiscale,Catégorie commune dans aire d'attraction des villes 2020,Tranche détaillée d'aire d'attraction des villes 2020,Libellé degré de densité,APL
0,9,14,0.092860,11.410345,8.743103,9.268966,9.30142,117,11483.5,46.153721,4.925850,en croissance démographique,Com rurale < 2 000 m habts,1.Accroissement par excédent naturel et migrat...,Bassin Industriel en croissance démographique,Fiscalité moyenne,Commune de la couronne,Aire de moins de 10 000 habitants,Rural à habitat dispersé,2.293
1,31,36,0.099229,11.410345,8.743103,9.268966,9.30142,110,11483.5,46.009606,5.428088,en croissance démographique,Com rurale < 2 000 m habts,1.Accroissement par excédent naturel et migrat...,Bassin Résidentiel en croissance démographique,Fiscalité moyenne,Commune hors attraction des pôles,Commune hors attraction des villes,Rural à habitat dispersé,2.600
2,31,36,0.099229,11.410345,8.743103,9.268966,9.30142,250,11483.5,45.961049,5.372275,en croissance démographique,Com < 50 m habts,1.Accroissement par excédent naturel et migrat...,Bassin Résidentiel en croissance démographique,Fiscalité moyenne,Commune-centre,Aire de 30 000 à moins de 50 000 habitants,Centres urbains intermédiaires,4.079
3,12,12,1.000000,11.410345,8.743103,9.268966,9.30142,127,11483.5,45.996164,4.911967,en croissance démographique,Com rurale < 2 000 m habts,Grande Ville,Bassin Urbain en croissance démographique,Fiscalité moyenne,Commune de la couronne,Aire de 1 000 000 d’habitants ou plus (hors Pa...,Bourgs ruraux,4.378
4,26,21,0.100905,11.410345,8.743103,9.268966,9.30142,109,11483.5,45.749886,5.594585,en croissance démographique,Com rurale < 2 000 m habts,3.Accroissement par excédent migratoire,Bassin Résidentiel en croissance démographique,Fiscalité moyenne,Commune de la couronne,Aire de 20 000 à moins de 30 000 habitants,Rural à habitat dispersé,1.069


In [8]:
# Select only numerical columns for the correlation matrix
numeric_columns = insee_df.select_dtypes(include=['float64', 'int64'])

# # Calculate the correlation matrix
corr_matrix = numeric_columns.corr().round(2)

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                   x=corr_matrix.columns.tolist(),
                                   y=corr_matrix.index.tolist())

fig.update_layout(height=800, width=1200)
fig.show()

In [9]:
# X, y split 
X = insee_df.loc[:, insee_df.columns != "APL"]
y = insee_df.loc[:, "APL"]

# X = X.select_dtypes(exclude=["object"])


In [10]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i,t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
    else :
        categorical_features.append(i)

print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Nb Omnipraticiens BV', 'Nb Infirmiers Libéraux BV', 'Densité Médicale BV', 'Dep Moyenne Salaires Horaires', 'Dep Moyenne Salaires Employé Horaires', 'Dep Moyenne Salaires Ouvrié Horaires', 'Reg Moyenne Salaires Ouvrié Horaires', 'Capacité Fisc', 'Moyenne Revnus fiscaux', 'latitude', 'longitude']
Found categorical features  ['SEG Croissance POP', 'Urbanité Ruralité', 'Dynamique Démographique BV', 'Environnement Démographique', 'Seg Cap Fiscale', "Catégorie commune dans aire d'attraction des villes 2020", "Tranche détaillée d'aire d'attraction des villes 2020", 'Libellé degré de densité']


In [11]:
# Train_test_split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)


In [12]:
#categorical_transformer = OneHotEncoder(drop='first')
from sklearn.pipeline import Pipeline

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))])

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [13]:
# Fit and transform X_train
X_train = preprocessor.fit_transform(X_train)
# Apply on X_test
X_test = preprocessor.transform(X_test)

# Visualize X_std_train
X_train

array([[-0.13680656,  0.29808188, -0.40964768, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.93601412,  1.47249275, -0.59979081, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.17027798,  0.71257984, -0.54571341, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.90451793, -0.66908001, -0.63206281, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.39664094,  2.64690362, -0.58670757, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.09350685, -0.18549906, -0.59717416, ...,  0.        ,
         1.        ,  0.        ]])

In [14]:
# Instanciate RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

# Print R^2 scores
print("R2 score on training set : ", rf.score(X_train, y_train))
print("R2 score on test set : ", rf.score(X_test, y_test))

R2 score on training set :  0.925221726491648
R2 score on test set :  0.4862074284484196


In [ ]:
#Score with all well cleaned Variables 
#R2 score on training set :  0.912866883506766
#R2 score on test set :  0.40418689908739636

In [ ]:
#Score with NO cleaned Variables
#R2 score on training set :  0.9243368812005699
#R2 score on test set :  0.45301035320888927

In [15]:
rf_best_Hyper_Param = RandomForestRegressor(max_depth = 18, min_samples_leaf = 2, min_samples_split= 2, n_estimators = 80)
rf_best_Hyper_Param.fit(X_train, y_train)

# Print R^2 scores
print("R2 score on training set : ", rf_best_Hyper_Param.score(X_train, y_train))
print("R2 score on test set : ", rf_best_Hyper_Param.score(X_test, y_test))

R2 score on training set :  0.7578663401318112
R2 score on test set :  0.45810398249483375


In [ ]:
from sklearn.decomposition import PCA

# Instanciate PCA with 3 components
pca = PCA(n_components=3)

# Fit transform X_std_train
X_opt_train = pca.fit_transform(X_train)

# Apply on X_std_test
X_opt_test = pca.transform(X_test)

In [ ]:
PC1 = X_opt_train[:, 0]
PC2 = X_opt_train[:, 1]
PC3 = X_opt_train[:, 2]

# Convert PC into a DataFrame
PC = pd.DataFrame(data=X_opt_train, columns=["PC1", "PC2", "PC3"])
# PC Head
PC.head()

In [ ]:
# Use pca.explained_variance_ratio_
print("Explained Variance ration per PC: {}".format(pca.explained_variance_ratio_))
print("Total explained variance ratio: {}%".format(pca.explained_variance_ratio_.sum()))

In [ ]:
# Fit the RF bestfeatures on the train set where the PCA was applied and checkout the score on the test
rf_best_Hyper_Param.fit(X_opt_train, y_train)
# Print R^2 scores
print("R2 score on training set fit on PCA: ", rf_best_Hyper_Param.score(X_opt_train, y_train))
print("R2 score on test set fit on PCA: ",     rf_best_Hyper_Param.score(X_opt_test, y_test))

In [ ]:
# NO IMPROVEMENTS ON SCORES WHEN MODEL FITTED ON PCA, EVEN WORSE SCORES

In [16]:
# Perform grid search
rf_best = RandomForestRegressor()#max_depth = 60, min_samples_leaf = 2, min_samples_split= 2, n_estimators = 220)

# print("Grid search...")
# # Grid of values to be tested
params = {
      'max_depth': [60],
      'min_samples_split': [2],
      'n_estimators': [200],
      'min_samples_leaf': [2] }

gridsearch = GridSearchCV(rf_best, param_grid= params, cv = 3, verbose = 2) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, y_train)
#rf_best.fit(X_train, y_train)
# print("...Done.")

print("Best hyperparameters :     ", gridsearch.best_params_)
print("Best validation accuracy : ", gridsearch.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END max_depth=60, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=  27.0s
[CV] END max_depth=60, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=  29.2s
[CV] END max_depth=60, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=  29.8s
Best hyperparameters :      {'max_depth': 60, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Best validation accuracy :  0.42676060527665927


In [17]:
#print("R2 score on training set : ", rf_best.score(X_train, y_train))
#print("R2 score on test set : ",     rf_best.score(X_test, y_test))
print("R2 score on training set : ", gridsearch.score(X_train, y_train))
print("R2 score on test set : ",     gridsearch.score(X_test, y_test))
print("Standard-deviation on test set : ", y_test.std())

R2 score on training set :  0.8612943527426964
R2 score on test set :  0.48424083482653124
Standard-deviation on test set :  1.2669167974941518


In [ ]:
#Score with the 20/30 best features  With GS
#Best hyperparameters :  {'max_depth': 100, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 260}
#Best validation accuracy :  0.42807176255229845

#R2 score on training set :  0.8616047925032368
#R2 score on test set :  0.4855002995513865
#Standard-deviation on test set :  1.2669167974941518

In [ ]:
#Score with the 20/30 best features  With GS
#Best hyperparameters :  {'max_depth': 60, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
#Best validation accuracy :  0.4275455441952625

#R2 score on training set :  0.8609884067130333
#R2 score on test set :  0.4874211365261111
#Standard-deviation on test set :  1.2669167974941518

In [ ]:
#Score with the 20/30 best features  With GS
#Best hyperparameters :  {'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 160}
#Best validation accuracy :  0.4272485432380593

#R2 score on training set :  0.8592108291179771
#R2 score on test set :  0.48489743908005256
#Standard-deviation on test set :  1.2669167974941518

In [ ]:
#Score with the 20/30 best features  With GS
#Best hyperparameters :  {'max_depth': 22, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 120}
#Best validation accuracy :  0.42143387449054975

#R2 score on training set :  0.8277264057371088
#R2 score on test set :  0.47848297028625475
#Standard-deviation on test set :  1.2669167974941518

In [ ]:
#Score with ALL well cleaned features With GS
#Best hyperparameters :  {'max_depth': 18, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 80}
#R2 score on training set :  0.8038804532618741
#R2 score on test set :  0.40011368458260765
#Standard-deviation on test set :  1.2669167974941518

In [18]:
column_names = []
for name, step, features_list in preprocessor.transformers_: # loop over steps of ColumnTransformer
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = step.get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Nb Omnipraticiens BV', 'Nb Infirmiers Libéraux BV', 'Densité Médicale BV', 'Dep Moyenne Salaires Horaires', 'Dep Moyenne Salaires Employé Horaires', 'Dep Moyenne Salaires Ouvrié Horaires', 'Reg Moyenne Salaires Ouvrié Horaires', 'Capacité Fisc', 'Moyenne Revnus fiscaux', 'latitude', 'longitude', 'SEG Croissance POP_en déclin démographique', 'Urbanité Ruralité_Com < 200 m habts', 'Urbanité Ruralité_Com < 50 m habts', 'Urbanité Ruralité_Com > 200 m habts', 'Urbanité Ruralité_Com rurale < 2 000 m habts', 'Urbanité Ruralité_Com rurale > 2 000 habts', 'Dynamique Démographique BV_2.Accroissement par excédent naturel', 'Dynamique Démographique BV_3.Accroissement par excédent migratoire', 'Dynamique Démographique BV_4.Déclin par déficit naturel et migratoire', 'Dynamique Démographique BV_5.Déclin par déficit naturel', 'Dynamique Démographique BV_6.Déclin par déficit migratoire', 'Dynamique Démographique BV_Grande Ville', 'Environnement Dém

In [19]:
# Create a pandas DataFrame
feature_importance = pd.DataFrame(index = column_names, data = gridsearch.best_estimator_.feature_importances_, columns=["feature_importances"])
feature_importance = feature_importance.sort_values(by = 'feature_importances')
feature_importance

,feature_importances
Environnement Démographique_Bassin diversifié en déclin démographique,0.000000
SEG Croissance POP_en déclin démographique,0.000000
Environnement Démographique_Bassin Résidentiel en déclin démographique,0.000002
Catégorie commune dans aire d'attraction des villes 2020_Commune d'un pôle secondaire,0.000060
Seg Cap Fiscale_Fiscalité élevée,0.000086
Catégorie commune dans aire d'attraction des villes 2020_Commune-centre,0.000255
Seg Cap Fiscale_Fiscalité moyenne,0.000510
Libellé degré de densité_Centres urbains intermédiaires,0.000716
Environnement Démographique_Bassin diversifié en croissance démographique,0.000790
Urbanité Ruralité_Com < 50 m habts,0.000808


In [20]:
Situation_Géographique = (feature_importance.loc['latitude'] + feature_importance.loc['longitude'] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 10 000 à moins de 20 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Commune hors attraction des villes"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 200 000 à moins de 300 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de moins de 10 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 50 000 à moins de 75 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 20 000 à moins de 30 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 75 000 à moins de 100 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 30 000 à moins de 50 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de Paris"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 300 000 à moins de 400 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 100 000 à moins de 125 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 400 000 à moins de 500 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 125 000 à moins de 150 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 150 000 à moins de 200 000 habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 700 000 à moins de 1 000 000 d’habitants"] + feature_importance.loc["Tranche détaillée d'aire d'attraction des villes 2020_Aire de 500 000 à moins de 700 000 habitants"])/18
Densité_Commune        = (feature_importance.loc["Libellé degré de densité_Rural à habitat très dispersé"] + feature_importance.loc["Urbanité Ruralité_Com rurale < 2 000 m habts"] + feature_importance.loc["Libellé degré de densité_Rural à habitat dispersé"] + feature_importance.loc["Libellé degré de densité_Grands centres urbains"] + feature_importance.loc["Libellé degré de densité_Ceintures urbaines"] + feature_importance.loc["Urbanité Ruralité_Com rurale > 2 000 habts"] + feature_importance.loc["Urbanité Ruralité_Com < 200 m habts"] + feature_importance.loc["Libellé degré de densité_Petites villes"] + feature_importance.loc["Urbanité Ruralité_Com > 200 m habts"] + feature_importance.loc["Urbanité Ruralité_Com < 50 m habts"] + feature_importance.loc["Libellé degré de densité_Centres urbains intermédiaires"])/11
Démographie            = (feature_importance.loc["Environnement Démographique_Bassin Résidentiel en croissance démographique"] + feature_importance.loc["Dynamique Démographique BV_3.Accroissement par excédent migratoire"] +  feature_importance.loc["Dynamique Démographique BV_5.Déclin par déficit naturel"] +  feature_importance.loc["Dynamique Démographique BV_4.Déclin par déficit naturel et migratoire"] +  feature_importance.loc["Dynamique Démographique BV_6.Déclin par déficit migratoire"] + feature_importance.loc["Dynamique Démographique BV_2.Accroissement par excédent naturel"] + feature_importance.loc["Environnement Démographique_Bassins Agroalimentaire en croissance démographique"] + feature_importance.loc["Dynamique Démographique BV_Grande Ville"] + feature_importance.loc["Environnement Démographique_Bassin Urbain en croissance démographique"] + feature_importance.loc["Environnement Démographique_Bassin diversifié en croissance démographique"] + feature_importance.loc["SEG Croissance POP_en déclin démographique"] + feature_importance.loc["Environnement Démographique_Bassin diversifié en déclin démographique"] + feature_importance.loc["Environnement Démographique_Bassin Résidentiel en déclin démographique"])/13
Offre_Médicale         = (feature_importance.loc["Densité Médicale BV"] + feature_importance.loc["Nb Omnipraticiens BV"] + feature_importance.loc["Nb Infirmiers Libéraux BV"])/3
Revenus                = (feature_importance.loc["Dep Moyenne Salaires Employé Horaires"] + feature_importance.loc["Moyenne Revnus fiscaux"] + feature_importance.loc["Dep Moyenne Salaires Ouvrié Horaires"] + feature_importance.loc["Dep Moyenne Salaires Horaires"] + feature_importance.loc["Reg Moyenne Salaires Ouvrié Horaires"] + feature_importance.loc["Seg Cap Fiscale_Fiscalité moyenne"] + feature_importance.loc["Seg Cap Fiscale_Fiscalité élevée"])/7


In [21]:
print("Importance de la situation géographique :   ", Situation_Géographique)
print("Importance de la densité communale :        ", Densité_Commune)
print("Importance de la démographie :              ", Démographie)
print("Importance de l'offre médicale :            ", Offre_Médicale)
print("Importance des revenus :                    ", Revenus)

Importance de la situation géographique :    feature_importances    0.027185
dtype: float64
Importance de la densité communale :         feature_importances    0.012029
dtype: float64
Importance de la démographie :               feature_importances    0.002203
dtype: float64
Importance de l'offre médicale :             feature_importances    0.05041
dtype: float64
Importance des revenus :                     feature_importances    0.012871
dtype: float64


In [22]:
import pandas as pd

data = [
    ["Situation_Géographique", 0.027186],
    ["Densité_Commune", 0.012028],
    ["Démographie", 0.002199],
    ["Offre_Médicale", 0.050863],
    ["Revenus", 0.012698]]

columns = ['Catégories', 'Importance']

df = pd.DataFrame(data, columns=columns)
print(df)

               Catégories  Importance
0  Situation_Géographique    0.027186
1         Densité_Commune    0.012028
2             Démographie    0.002199
3          Offre_Médicale    0.050863
4                 Revenus    0.012698


In [23]:
# Plot coefficients
fig = px.bar(df, x = "Importance", y = "Catégories", title="APL et Ordre d'importance des Indicateurs", orientation = 'h')
#fig.update_layout(height=100, width=500, showlegend = False, margin = {'l': 50}, )
fig.show()

In [24]:
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(height=1000, width=1500, showlegend = False, margin = {'l': 50}, ) # to avoid cropping of column names
fig.show()